In [1]:
# all codes come are changed based on LDR code.
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# data_preprocessing and parameters

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 0
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)


In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
num_steps = 150
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG = get_models(DATASET, device)

optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []




transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
netD, netG

(DataParallel(
   (module): DiscriminatorMNIST(
     (main): Sequential(
       (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (1): LeakyReLU(negative_slope=0.2, inplace=True)
       (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (4): LeakyReLU(negative_slope=0.2, inplace=True)
       (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
       (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (7): LeakyReLU(negative_slope=0.2, inplace=True)
       (8): Conv2d(256, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (9): Flatten(start_dim=1, end_dim=-1)
     )
   )
 ),
 DataParallel(
   (module): GeneratorMNIST(
     (main): Sequential(
       (0): ConvTranspose2d(128, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
       (1)

In [6]:
real_cpu.shape

torch.Size([2048, 1, 32, 32])

In [7]:
netD(real_cpu).shape

torch.Size([2048, 128])

In [8]:
netG(netD(real_cpu)).shape

torch.Size([2048, 1, 32, 32])

In [9]:
netD(netG(netD(real_cpu))).shape

torch.Size([2048, 128])

In [10]:
netD(netG(netD(real_cpu)))

tensor([[ 0.0484,  0.0411,  0.0302,  ..., -0.0061, -0.0961,  0.1603],
        [ 0.0047,  0.1078,  0.0183,  ...,  0.1106,  0.0581,  0.1612],
        [ 0.0423,  0.1187,  0.1185,  ...,  0.0034,  0.0579,  0.0528],
        ...,
        [ 0.0626,  0.1588,  0.0691,  ..., -0.0503,  0.1613,  0.0436],
        [ 0.0042,  0.0262, -0.0174,  ...,  0.2025,  0.0006,  0.1767],
        [ 0.0407, -0.0084,  0.0065,  ...,  0.0680,  0.1381,  0.0634]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [11]:
# fixed radius case, due to this requires more steps to go to about -200 then set 1500 epoches

n_dis = 1
n_dis2 = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)

num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 0

for epoch in range(num_steps):
    for i in range(n_dis):
        netD.zero_grad()
        optD.zero_grad()
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
        print(errD, errD_EC)
        errD.backward()
        optD.step()
        
    for j in range(n_dis2):
        netG.zero_grad()
        optG.zero_grad()
        
        Z = netD(real_cpu)
        fake_img = netG(netD(real_cpu)).detach()
        Z_bar = netD(fake_img)
        
        errG, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis2)
        print(errG, errD_EC)
        errG = - errG
        errG.backward()
        optG.step()

    
    

tensor(-73.8114, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-9.4169, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4860, device='cuda:0', grad_fn=<NegBackward0>), tensor(-55.9085, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-40.3671, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3124, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.9134, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.1414, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-40.3671, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-4.3124, device='cuda:0', grad_fn=<NegBackward0>), tensor(-3.9134, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.1413, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-55.2940, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-6.1767, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.7523, device='cuda:0', grad_fn=<NegBackward0>), tensor(-43.3650, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-55.2941, device='cuda:0', grad_fn=<NegBackward0>) [tenso

tensor(-132.2291, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.4492, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.3640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-106.4158, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-132.2291, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.4492, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.3640, device='cuda:0', grad_fn=<NegBackward0>), tensor(-106.4159, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.2773, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.7261, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.2359, device='cuda:0', grad_fn=<NegBackward0>), tensor(-108.3153, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.2773, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.7261, device='cuda:0', grad_fn=<NegBackward0>), tensor(-11.2359, device='cuda:0', grad_fn=<NegBackward0>), tensor(-108.3153, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-135.0098, device='cuda:0', grad_fn=<NegB

tensor(-156.7446, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.8360, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.4126, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.4960, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-157.5195, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9288, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.4602, device='cuda:0', grad_fn=<NegBackward0>), tensor(-128.1306, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-157.5195, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-16.9288, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.4602, device='cuda:0', grad_fn=<NegBackward0>), tensor(-128.1306, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-156.4774, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.0693, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.4053, device='cuda:0', grad_fn=<NegBackward0>), tensor(-127.0028, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-156.4774, device='cuda:0', grad_fn=<NegB

tensor(-165.4131, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.8438, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.5820, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.9873, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-165.4131, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.8438, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.5820, device='cuda:0', grad_fn=<NegBackward0>), tensor(-134.9873, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.3983, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.8526, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.6554, device='cuda:0', grad_fn=<NegBackward0>), tensor(-135.8903, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.3983, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-17.8526, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.6554, device='cuda:0', grad_fn=<NegBackward0>), tensor(-135.8903, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-166.8206, device='cuda:0', grad_fn=<NegB

tensor(-169.7750, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.4996, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.9017, device='cuda:0', grad_fn=<NegBackward0>), tensor(-138.3736, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-170.8739, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.6407, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.9944, device='cuda:0', grad_fn=<NegBackward0>), tensor(-139.2388, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-170.8738, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.6407, device='cuda:0', grad_fn=<NegBackward0>), tensor(-12.9944, device='cuda:0', grad_fn=<NegBackward0>), tensor(-139.2388, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.8516, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.6955, device='cuda:0', grad_fn=<NegBackward0>), tensor(-13.0862, device='cuda:0', grad_fn=<NegBackward0>), tensor(-140.0699, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-171.8516, device='cuda:0', grad_fn=<NegB

...

tensor(-197.9748, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2666, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.9459, device='cuda:0', grad_fn=<NegBackward0>), tensor(-147.7624, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-198.6216, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2287, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.0882, device='cuda:0', grad_fn=<NegBackward0>), tensor(-148.3046, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-198.6216, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.2287, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.0882, device='cuda:0', grad_fn=<NegBackward0>), tensor(-148.3046, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.7319, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.5780, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.4594, device='cuda:0', grad_fn=<NegBackward0>), tensor(-149.6945, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-200.7318, device='cuda:0', grad_fn=<NegB

tensor(-190.6937, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.6110, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.3447, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.7380, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-190.6937, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-28.6110, device='cuda:0', grad_fn=<NegBackward0>), tensor(-20.3447, device='cuda:0', grad_fn=<NegBackward0>), tensor(-141.7380, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-195.6107, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.1259, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.0777, device='cuda:0', grad_fn=<NegBackward0>), tensor(-145.4071, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-195.6107, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-29.1259, device='cuda:0', grad_fn=<NegBackward0>), tensor(-21.0777, device='cuda:0', grad_fn=<NegBackward0>), tensor(-145.4071, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-196.0177, device='cuda:0', grad_fn=<NegB

tensor(-212.6741, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.1991, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.0166, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.4584, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-212.3431, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.1853, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.0074, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.1505, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-212.3431, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.1853, device='cuda:0', grad_fn=<NegBackward0>), tensor(-24.0073, device='cuda:0', grad_fn=<NegBackward0>), tensor(-157.1504, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-210.8409, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.1547, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.7835, device='cuda:0', grad_fn=<NegBackward0>), tensor(-155.9027, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-210.8409, device='cuda:0', grad_fn=<NegB

tensor(-210.7454, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.2431, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.8179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-155.6844, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-210.7452, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.2431, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.8179, device='cuda:0', grad_fn=<NegBackward0>), tensor(-155.6842, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-211.6486, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.3341, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.9873, device='cuda:0', grad_fn=<NegBackward0>), tensor(-156.3272, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-211.6486, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.3341, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.9873, device='cuda:0', grad_fn=<NegBackward0>), tensor(-156.3272, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-212.0648, device='cuda:0', grad_fn=<NegB

tensor(-204.2794, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-30.7501, device='cuda:0', grad_fn=<NegBackward0>), tensor(-22.6843, device='cuda:0', grad_fn=<NegBackward0>), tensor(-150.8451, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-206.5897, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0255, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.0670, device='cuda:0', grad_fn=<NegBackward0>), tensor(-152.4972, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-206.5897, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.0255, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.0670, device='cuda:0', grad_fn=<NegBackward0>), tensor(-152.4972, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-208.6432, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-31.2173, device='cuda:0', grad_fn=<NegBackward0>), tensor(-23.4337, device='cuda:0', grad_fn=<NegBackward0>), tensor(-153.9922, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-208.6432, device='cuda:0', grad_fn=<NegB

In [12]:
#acc at first for NetD
NetD_acc = 0
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
        
NetD_acc = count/10000
print('NetD, Acc at first', NetD_acc)
Acc_results_D.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]


----------------------------
Train_z, Test_z
PCA: 0.9344
SVD: 0.9214
NetD, Acc at first 0.964
